# Parkinson’s Mouse Tutorial

**_note:_ This guide assumes you have QIIME 2 installed (e.g. using this [procedure](https://docs.qiime2.org/2019.10/install/native/)). To execute the script properly, open this notebook in a Jupyter Notebook from within a conda QIIME 2 environment.**

**_note:_ This tutorial is an adaptation of the same tutorial that may be found on the [official QIIME 2 docs website](https://docs.qiime2.org/2020.2/tutorials/atacama-soils/). The original tutorial uses the QIIME 2 CLI interface.**

Instead of CLI interface, this tutorial uses [Artifact API](https://docs.qiime2.org/2019.10/interfaces/artifact-api/) - a Python 3 application programmer interface (API) for QIIME 2. The Artifact API supports interactive computing with QIIME 2 using the Python 3 programming language. The API is automatically generated, and its availability depends on which QIIME 2 plugins are currently installed. It has been optimized for use in the Jupyter Notebook. The Artifact API is a part of the QIIME 2 framework; no additional software needs to be installed to use it. 

**The notebook was tested using the `2020.2` version of QIIME 2.

This document is intended to be run after the [moving pictures tutorial](https://docs.qiime2.org/2020.8/tutorials/moving-pictures/). It is designed to introduce a few new ideas, and to be an exercise in applying the tools that were explored in that document. 
This tutorial will demonstrate a “typical” QIIME 2 analysis of 16S rRNA gene amplicon data, using a set of fecal samples from [humanized mice](https://en.wikipedia.org/wiki/Humanized_mouse). The original study, [Sampson et al, 2016](https://pubmed.ncbi.nlm.nih.gov/27912057/), was designed to determine whether the fecal microbiome contributed to the development of Parkinson’s Disease (PD). Several observation studies showed a difference in the microbiome between PD patients and controls, although the organisms identified across studies were not consistent. However, this was sufficient evidence to suggest that there might be a relationship between PD and the fecal microbiome.

To determine whether that relationship was incidental or actually disease associated, a second study was needed. A human cohort study was not feasible; the disease only affects about 1% of the population over 60 years old, PD takes a long time to develop and to be diagnosed, and it would be difficult to determine when to collect the samples. Therefore, a gnotobiotic mouse study was utilized to evaluate the role of the microbiome in the development of PD symptoms. Feces were collected from six donors with Parkinson’s disease and six age- and sex-matched neurologically health controls, and then transplanted into mice who were either predisposed to developing Parkinson’s disease due to a mutation (“aSyn”) or resistant wild type mice (“BDF1”). Mice from different donors were kept in separate cages, but mice from different genetic backgrounds were co-housed. The mice were followed for 7 weeks to see if they developed symptoms of Parkinson’s disease.

We’ll look a subset of data from two human donors (one healthy and one with PD) whose samples were each transplanted into three separate cages of mice from the susceptible genotype. For this tutorial, a subset of the metadata has been prepared, and the sequences have been subsampled to approximately 5000 sequences per sample to allow the tutorial to run in a short time. The sequences for the full study are accessible at EBI with accession [PRJEB17694](https://www.ebi.ac.uk/ena/data/view/PRJEB17694); processed tables from the full study can be downloaded from the [Qiita](https://qiita.ucsd.edu/) database from study 10483.

# Hypothesis

This tutorial will explore the hypothesis that the genetic background of a humanized mouse influences the microbial community. However, we’ll also need to consider other confounders which might drive the shape of the microbiome instead of the mouse genotype.

# Importing necessary modules

In [1]:
import qiime2

In [2]:
from qiime2.plugins import (demux, dada2, metadata, feature_table,
                            fragment_insertion, diversity, 
                            longitudinal, phylogeny, sample_classifier,
                            feature_classifier, taxa, composition, 
                            longitudinal)

# Set up

This tutorial assumes that you have QIIME 2 installed according to the [installation instructions](https://docs.qiime2.org/2020.6/install/).

Before running the tutorial, you will need to make a directory for the tutorial data and navigate into that directory.

In [3]:
workdir='/home/user/Documents/qiime2-mouse-tutorial/'

In [4]:
!mkdir -p $workdir
!cd $workdir

# Metadata

Before starting any analysis, it’s important to be familiar with the metadata. In this study, the metadata file contains 7 columns.

|        variable       |                                                                                        description                                                                                        |    data type    |                                              values                                              |
|:---------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:---------------:|:------------------------------------------------------------------------------------------------:|
|         sample        |                                                                                  unique sample identifier                                                                                 |        —        |                                      unique for each sample                                      |
|        mouse_id       |                                                                           the unique identifier  for each mouse                                                                           |    categorical  | `435` ; `437` ; `456` ; `457` ;  `468` ; `469` ; `536` ; `53`7 ;  `538` ; `539` ; `546` ; `547`; |
|          genotype     | the genetic background  of the mouse. The Thy1- aSyn (`susceptible`)  mice are genetically  predisposed to disease; `wild_type` from the BDF1  background do not have  any additional ris |     categorical |                                     `wild_type` ; `susceptible`                                  |
|        cage_id        |                                                                        the unique identifier  for each cage of mice                                                                       |   categorical   |                           `C31` ; `C35` ; `C42` ; `C43` ;  `C44`; `C49`                          |
|          donor        |                                                                 A unique identifier  for the human who  donated the feces                                                                 |    categorical  |                                          `hc_1`  `pd_1`                                          |
|       donor_status    |                               whether the donor  has Parkinson’s  disease or not (Donor `pd_1` had Parkinson’s  disease; `hc_1` was  neurologically healthy)                              |     categorical |                                          `Healthy` ; `PD`                                        |
|  days_post_transplant |                                                                     the number of days  after the mice were  humanized                                                                    |      numeric    |                                           7, 14, 21, 49                                          |

Even though the mouse ID looks like a number, we will specify that it is categorical using the `#q2_type directive`.

The metadata is available as a [Google Sheet](https://docs.google.com/spreadsheets/d/1SDBD5gYFy2ck_vZXdj2oBix5XCtnWMPVbs0aaLcdccw/edit?usp=sharing) , or you can download it directly and save it as a TSV (tab-separated values) file.

In [5]:
!wget -O $workdir/"metadata.tsv" \
  "https://data.qiime2.org/2019.10/tutorials/pd-mice/sample_metadata.tsv"

--2021-01-22 12:29:19--  https://data.qiime2.org/2019.10/tutorials/pd-mice/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/e/2PACX-1vRovoHFup2t5MxddVNLcJ96f5hkh5DJsx84uqCsTwAkz06i0ibrCI5Tn0xFF15xHDmVXfDYVb2o_YXD/pub?gid=1509704122&single=true&output=tsv [following]
--2021-01-22 12:29:20--  https://docs.google.com/spreadsheets/d/e/2PACX-1vRovoHFup2t5MxddVNLcJ96f5hkh5DJsx84uqCsTwAkz06i0ibrCI5Tn0xFF15xHDmVXfDYVb2o_YXD/pub?gid=1509704122&single=true&output=tsv
Resolving docs.google.com (docs.google.com)... 216.58.209.14, 2a00:1450:401b:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.209.14|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0o-6o-sheets.googleusercontent.com/pub/l5l039s6ni5uumqbsj9o11lmdc/4a187s1qu

# Importing data as a qiime2 artifact

All data that is used as input to QIIME 2 is in form of QIIME 2 artifacts, which contain information about the type of data and the source of the data. The first thing we need to do is import these sequence data files into a QIIME 2 artifact.

In [7]:
sample_metadata = qiime2.Metadata.load(workdir+'/metadata.tsv')

# Importing data into QIIME 2

All data that is used as input to QIIME 2 is in form of QIIME 2 artifacts, which contain information about the type of data and the source of the data. The first thing we need to do is import these sequence data files into a QIIME 2 artifact.

In QIIME 2, all data is structured as an Artifact of a specific semantic type. Artifacts contain the data as well as information about the data, including a record of the original data and the tools used to process it. This allows for better tracking of how you actually got to where you are in your analysis. You can learn more about common QIIME 2 Artifacts and semantic types [here](https://docs.qiime2.org/2020.8/semantic-types/).

Our samples were amplified using the [EMP 515f-806r](https://www.earthmicrobiome.org/protocols-and-standards/16s/) primers and sequenced on an Illumina MiSeq with a 2x150bp kit. The hypervariable region covered by the primers we used is 290bp long, so with 150bp reads our sequences will be slightly too short to be able to do paired-end analysis downstream. Therefore, we’re going to work with single-end sequences. We will work with a version of the samples which have already been demultiplexed, for example, by the sequencing center. If you need to demultiplex your sequences, the [Moving Pictures tutorial](https://docs.qiime2.org/2020.6/tutorials/moving-pictures/)describes how to demultiplex sequences if they were sequenced using the Earth Microbiome Project protocol.

We will import the sequences as `SampleData[SequencesWithQuality]`, which is the demultiplexed single-end sequence format. If we wanted to import paired-sequences, we would specify the semantic type `SampleData[PairedEndSequencesWithQuality]`. We will import the sequences using the sample [manifest format](https://docs.qiime2.org/2020.6/tutorials/importing/#manifest-file), a versatile way to import demultiplexed data in QIIME 2. We create a tab-separated sample manifest file that maps the sample name we want to use in QIIME 2 to the path of the sequence file. The benefit is that the demultiplexed sequence files can be named anything you want; there are not fixed assumptions about the conventions, and the file names do not dictate the final name. When QIIME 2 reads the file, it ignores any line prefixed with the # symbol. The first line that doesn’t contain a # is the header line and must be `sample-id<TAB>absolute-filepath`. The sample order after the header line does not matter. Read more about importing data into QIIME 2 artifacts [here](https://docs.qiime2.org/2020.6/tutorials/importing/) and more about sample metadata formatting requirements [here](https://docs.qiime2.org/2020.6/tutorials/metadata/).

Let’s start by downloading the manifest and corresponding sequences.

In [8]:
!wget -O $workdir/"manifest.tsv" \
  "https://data.qiime2.org/2019.10/tutorials/pd-mice/manifest"

--2021-01-22 12:29:30--  https://data.qiime2.org/2019.10/tutorials/pd-mice/manifest
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/tutorials/pd-mice/manifest [following]
--2021-01-22 12:29:30--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/tutorials/pd-mice/manifest
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.234.0
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.234.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4640 (4.5K) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//manifest.tsv’

/home/user/Document 100%[===================>]   4.53K  --.-KB/s    in 0s      

2021-01-22 12:29:31 (8.96 MB/s) - ‘/home/user/Documents/qiime2-mouse-tutorial//manifest.ts

In [9]:
!wget -O $workdir/"demultiplexed_seqs.zip" \
  "https://data.qiime2.org/2019.10/tutorials/pd-mice/demultiplexed_seqs.zip"

--2021-01-22 12:29:33--  https://data.qiime2.org/2019.10/tutorials/pd-mice/demultiplexed_seqs.zip
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/tutorials/pd-mice/demultiplexed_seqs.zip [following]
--2021-01-22 12:29:34--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/tutorials/pd-mice/demultiplexed_seqs.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.217.104
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.217.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21508775 (21M) [application/zip]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//demultiplexed_seqs.zip’

/home/user/Document 100%[===================>]  20.51M   268KB/s    in 60s     

2021-01-22 12:30:35 (350 KB/s) - ‘/

In [10]:
!unzip demultiplexed_seqs.zip

unzip:  cannot find or open demultiplexed_seqs.zip, demultiplexed_seqs.zip.zip or demultiplexed_seqs.zip.ZIP.


Importing manigest as a qiime2 artifact

In [11]:
manifest = qiime2.Metadata.load(workdir+'/manifest.tsv')

When using this manifest format, a sample name can only appear in one line and can only map to one sequencing file per column (one column for single-end, two columns for paired-end). The absolute-filepath for each sample must be an [absolute path](https://en.wikipedia.org/wiki/Path_(computing)#Absolute_and_relative_paths), which specifies the “complete” location of the file. We do that here using the `$PWD` variable, which expands the current directory in absolute terms.

We’ll use the manifest to import our data.

In [12]:
manifest.to_dataframe().head()

,absolute-filepath
sample-id,
recip.220.WT.OB1.D7,$PWD/demultiplexed_seqs/10483.recip.220.WT.OB1...
recip.290.ASO.OB2.D1,$PWD/demultiplexed_seqs/10483.recip.290.ASO.OB...
recip.389.WT.HC2.D21,$PWD/demultiplexed_seqs/10483.recip.389.WT.HC2...
recip.391.ASO.PD2.D14,$PWD/demultiplexed_seqs/10483.recip.391.ASO.PD...
recip.391.ASO.PD2.D21,$PWD/demultiplexed_seqs/10483.recip.391.ASO.PD...


In [13]:
demux_seqs = qiime2.Artifact.import_data(type='SampleData[SequencesWithQuality]',
                                         view_type='SingleEndFastqManifestPhred33V2',
                                         view=workdir+'/manifest.tsv')

Let’s check the sequences and the sequencing depth of the samples using the `qiime demux summarize command`. It provides information about the number of sequences in each sample, as well as the quality of the sequences.

Before running the command, let’s review the help documentation to make sure we understand the arguments for the command.

In [14]:
!qiime demux summarize --help

Usage: qiime demux summarize [OPTIONS]

  Summarize counts per sample for all samples, and generate interactive
  positional quality plots based on `n` randomly selected sequences.

Inputs:
  --i-data ARTIFACT SampleData[SequencesWithQuality |
    PairedEndSequencesWithQuality | JoinedSequencesWithQuality]
                       The demultiplexed sequences to be summarized.
                                                                    [required]
Parameters:
  --p-n INTEGER        The number of sequences that should be selected at
                       random for quality score plots. The quality plots will
                       present the average positional qualities across all of
                       the sequences selected. If input sequences are paired
                       end, plots will be generated for both forward and
                       reverse reads for the same `n` sequences.
                                                              [default: 10000]
Outputs:

Based on the documentation, we should specify the file (Artifact) with the demultiplexed sequences for the `--i-`data argument, since this expects data of semantic type `SampleData[SequencesWithQuality]`. We’ll specify the location we want to save the visualization to by specifying the output path to `--o-visualization`.

The help documentation is a good reference for any command, and the first place to look if you’re getting errors, especially errors about parameters.

In [15]:
demux_summary = demux.visualizers.summarize(demux_seqs)
demux_summary.visualization

<visualization: Visualization uuid: ba2f6b27-c51d-406a-b26b-79eecab3716f>

### Question

1. After demultiplexing, which sample has the lowest sequencing depth?

2. What is the median sequence length?

3. What is the median quality score at position 125?

4. If you are working on this tutorial alongside someone else, why does your plot look slightly different from your neighbors? If you aren’t working alongside someone else, try running this command a few times and compare the results.

### Checkpoint

What are good positions to consider trimming and/or truncating at?

# Sequence quality control and feature table

There are several ways to construct a feature table in QIIME 2. The first major choice to make is to work with Operational Taxonomic Units (OTUs) or Absolute Sequence Variants (ASVs). OTUs have been widely used in microbiome research since the mid 2010s, and assign sequences to clusters either based on a reference database or de novo assignment. QIIME 2 offers clustering through [q2-vsearch](https://docs.qiime2.org/2020.6/tutorials/otu-clustering/) and [q2-dbOTU](https://library.qiime2.org/plugins/q2-dbotu/4/) plug-ins, currently.

ASVs are a more recent development and provide better resolution in features than traditional OTU-based methods. ASVs can separate features based on differences of a single nucleotide in sequences of 400 bp or more, a resolution not possibly even with 99% identity OTU clustering. QIIME 2 currently offers denoising via [DADA2](https://pubmed.ncbi.nlm.nih.gov/27214047/) (dada2) and [Deblur](https://pubmed.ncbi.nlm.nih.gov/28289731/) (deblur). The major differences in the algorithms and motivation for denoising are nicely described in [Nearing et al, 2018](https://pubmed.ncbi.nlm.nih.gov/30123705/).

It is worth noting in either case that denoising to ASVs and clustering to OTUs are separate, but parallel steps. A choice should be made for a single pathway: either denoising or OTU based clustering; it is not recommended to combine the steps.

In this tutorial, we’ll denoise with DADA2 (using single-end sequences). Please see the [Atacama Soil tutorial](https://docs.qiime2.org/2020.6/tutorials/atacama-soils/) for an example of using DADA2 on paired-end sequences. For those interested in using Deblur, you can refer to the [Moving Pictures](https://docs.qiime2.org/2020.6/tutorials/moving-pictures/) tutorial and [Alternative methods of read joining](https://docs.qiime2.org/2020.6/tutorials/read-joining/) tutorial for running Deblur on single- and paired-end sequences, respectively.

The qiime `dada2.methods.denoise_single` method requires us to set the `trunc_len` parameter. This controls the length of the sequences and should be selected based on a drop in quality scores. In our dataset, the quality scores are relatively evenly distributed along the sequencing run, so we’ll use the full 150 bp sequences. However, the selection of the trim length is a relatively subjective measurement and relies on the decision making capacity of the analyst.

In [18]:
denoised_sequences = dada2.methods.denoise_single(demux_seqs,
                                                  trunc_len = 150)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-jaxx0wn2/45ecc05a-c828-43cf-bfcd-376df34cceef/data /tmp/tmpllge5vsd/output.tsv.biom /tmp/tmpllge5vsd/track.tsv /tmp/tmpllge5vsd 150 0 2.0 2 Inf consensus 1.0 1 1000000 NULL 16



We can also review the denoising statistics using the `metadata.visualizers.tabulate` command.



In [19]:
filter_stats = metadata.visualizers.tabulate(denoised_sequences.denoising_stats.view(qiime2.Metadata))
filter_stats.visualization

<visualization: Visualization uuid: 61e74f44-108a-4639-85e2-c39a7fac1cc2>

## Feature table summary

After we finish denoising the data, we can check the results by looking at the summary of the feature table. This will provide us with the counts associated with each sequence and each feature, as well as other useful plots and metrics.

In [20]:
output_viz = feature_table.visualizers.summarize(denoised_sequences.table)
output_viz.visualization

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: b8a58e67-252b-4b6b-b0b1-f52fd7425716>

### Question

1. How many total features remain after denoising?

2. Which sample has the highest total count of features? How many sequences did that sample have prior to DADA2 denoising?

3. How many samples have fewer than 4250 total features?

4. Which features are observed in at least 47 samples?

5. Which sample has the fewest features? How many does it have?

If you open the denoising summary, can you find the step where the sample with the fewest sequences fails?

# Generating a phylogenetic tree for diversity analysis

QIIME 2 analysis allows the use of phylogenetic trees for diversity metrics such as Faith’s Phylogenetic Diversity and UniFrac distance as well as feature-based analyses in Gneiss. The tree provides an inherent structure to the data, allowing us to consider an evolutionary relationship between organisms.

QIIME 2 offers several ways to construct a phylogenetic tree. For this tutorial, we’re going to create a fragment insertion tree using the `fragment_insertion` plugin. The authors of the fragment insertion plugin suggest that it can outperform traditional alignment based methods based on short Illumina reads by alignment against a reference tree built out of larger sequences. Our command, qiime `fragment_insertion.methods.sepp` will use the representative sequences (a `FeatureData[Sequence]` artifact) we generated during denoising to create a phylogenetic tree where the sequences have been inserted into the greengenes 13_8 99% identity reference tree backbone.

First, we will download the reference database:

In [21]:
!wget -O $workdir/"sepp-refs-gg-13-8.qza" \
  "https://data.qiime2.org/2019.10/common/sepp-refs-gg-13-8.qza"

--2020-09-06 18:13:25--  https://data.qiime2.org/2019.10/common/sepp-refs-gg-13-8.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/common/sepp-refs-gg-13-8.qza [following]
--2020-09-06 18:13:25--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.10/common/sepp-refs-gg-13-8.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.208.208
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.208.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50161069 (48M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//sepp-refs-gg-13-8.qza’

/home/user/Document 100%[===================>]  47.84M  1.91MB/s    in 43s     

2020-09-06 18:14:10 (1.10 MB/s) - ‘/home/user/Documents/qiime2-mouse

#### Importing the reference database as a qiime2 artifact

In [22]:
database_sepp = qiime2.Artifact.load(workdir+'/sepp-refs-gg-13-8.qza')

### Note

This command is resource intensive - if your computation environment supports it, we suggest including an appropriately-set `threads` parameter.

In [23]:
rooted_tree = phylogeny.pipelines.align_to_tree_mafft_fasttree(denoised_sequences.representative_sequences)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2-archive-m47z0cr8/a1615473-3a8e-47e0-981e-9f31f125e496/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-kbpq66ig/e59d8778-5bd7-4c2a-9da6-8422e0c120f8/data/aligned-dna-sequences.fasta



In [24]:
tree_placements = fragment_insertion.methods.sepp(denoised_sequences.representative_sequences, 
                                                  database_sepp,
                                                  threads = 4)

# Alpha Rarefaction and Selecting a Rarefaction Depth

We now have a feature table (observation matrix) of ASVs in each sample, and a phylogenetic tree representing those ASVs, so are almost ready to perform various analyses of microbial diversity. However, first we must normalize our data to account for uneven sequencing depth between samples.

Although sequencing depth in a microbiome sample does not directly relate to the original biomass in a community, the relative sequencing depth has a large impact on observed communities [(Weiss et al, 2017)](https://pubmed.ncbi.nlm.nih.gov/28253908/). Therefore, for most diversity metrics, a normalization approach is needed.

Current best practices suggest the use of rarefaction, a normalization via sub-sampling without replacement. Rarefaction occurs in two steps: first, samples which are below the rarefaction depth are filtered out of the feature table. Then, all remaining samples are subsampled without replacement to get to the specified sequencing depth. It’s both important and sometimes challenging to select a rarefaction depth for diversity analyses. Several strategies exist to figure out an appropriate rarefaction depth - we will primarily consider alpha rarefaction in this tutorial, because it is a data-driven way to approach the problem.

We’ll use `diversity.visualizers.alpha_rarefaction` to subsample the ASV table at different depths (between `min_depth` and `max_depth`) and calculate the alpha diversity using one or more metrics . When we checked the feature table, we found that the sample with the fewest sequences in the denoised table has 85 features and that the sample with the most has 4996 features. We want to set a maximum depth close to the maximum number of sequences. We also know that if we look at a sequencing depth around 4250 sequences per sample, we’ll be looking at information from 22 samples. So, let’s set this as our maximum sequencing depth.

At each sampling depth, 10 rarefied tables are usually calculated to provide an error estimate, although this can be adjusted using the `iterations` parameter. We can check and see if there is a relationship between the alpha diversity and metadata by specifying the metadata file for the `metadata = sample_metadata` parameter.

In [25]:
normalized_data = diversity.visualizers.alpha_rarefaction(denoised_sequences.table, 
                                                          metadata = sample_metadata,
                                                          min_depth = 10,
                                                          max_depth = 4250)
                                                

In [27]:
normalized_data.visualization

<visualization: Visualization uuid: 5adb0b4c-c20d-4801-b532-7e6c5fdacb36>

The visualization file will display two plots. The upper plot will display the alpha diversity (observed OTUs or shannon) as a function of the sampling depth. This is used to determine whether the richness or evenness has saturated based on the sampling depth. The rarefaction curve should “level out” as you approach the maximum sampling depth. Failure to do so, especially with a diversity-only metric such as observed OTUs or Faith’s PD diversity, may indicate that the richness in the samples has not been fully saturated.

The second plot shows the number of samples in each metadata category group at each sampling depth. This is useful to determine the sampling depth where samples are lost, and whether this may be biased by metadata column group values. Remember that rarefaction is a two-step process and samples that do not meet the rarefaction depth are filtered out of the table. We can use the curves to look at the number of samples by different metadata columns.

If you’re still unsure of the rarefaction depth, you can also use the sample summary to look at which samples are lost by supplying sample metadata to the feature table summary.

### Question

Start by opening the alpha rarefaction visualization.

1. Are all metadata columns represented in the visualization? If not, which columns were excluded and why?

2. Which metric shows saturation and stabilization of the diversity?

3. Which mouse genetic background has higher diversity, based on the curve? Which has shallower sampling depth?

Now, let’s check the feature table summary.

4. What percentage of samples are lost if we set the rarefaction depth to 2500 sequences per sample?

5. Which mice did the missing samples come from?

After we’ve looked through the data, we need to select a rarefaction depth. In general, selecting a rarefaction depth is a subjective process that requires that analyst’s discretion. Selecting a rarefaction depth is an exercise in minimizing sequence loss while maximizing the sequences retained for diversity analysis. For high-biomass samples (fecal, oral, etc), a general best estimate is a rarefaction depth of no less than 1000 sequences per sample. In low biomass samples where sequencing is shallower, a lower rarefaction depth may be selected although it’s important to keep in mind that the diversity measurements on these samples will be quite noisy and the overall quality will be low.

### Checkpoint

Based on the current rarefaction curve and sample summary, what sequencing depth would you pick? Why?

In this case, we can retain 47 samples with a rarefaction depth of 2000 sequences/sample.

Based on the sequencing depth and distribution of samples, we’ll use 2000 sequences/sample for this analysis. This will let us keep 47 of 48 high quality samples (discarding the one sample with sequencing depth below 1000 sequences/sample).

# Diversity analysis

The first step in hypothesis testing in microbial ecology is typically to look at within- (alpha) and between-sample (beta) diversity. We can calculate diversity metrics, apply appropriate statistical tests, and visualize the data using the `diversity` plugin.

We’ll start by using the qiime `diversity.pipelines.core_metrics_phylogenetic` method, which ratifies the input feature table, calculates several commonly used alpha- and beta-diversity metrics, and produces principal coordinate analysis (PCoA) visualizations in Emperor for the beta diversity metrics. By default, the metrics computed are:

Alpha Diversity

- Shannon’s diversity index

- Observed Features

- Faith’s phylogenetic diversity

- Pielou’s evenness

Beta Diversity

- Jaccard distance

- Bray-Curtis distance

- Unweighted UniFrac distance

- Weighted UniFrac distance




### Note

🏗👷 Some descriptions are changing in QIIME 2’s diversity tools. The phrase “observed otus” is being replaced with “observed features”, because “features” better describes the different ways in which users work with non-taxonomic features. This will affect both documentation and (in places) the names of command arguments. You will see both phrases, but these measures of diversity are identical under the hood.

There is a very good discussion of diversity metrics and their meanings in a [forum post by Stephanie Orchanian](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282).

The qiime `diversity.pipelines.core_metrics_phylogenetic` method wraps several other methods, and it’s worthwhile to note that the steps can also be executed independently.

One important consideration for diversity calculations is the rarefaction depth. Above, we used the alpha rarefaction visualization and the sample summary visualization to pick a rarefaction depth. So, for these analyses, we’ll use a depth of 2000 sequences per sample.

In [28]:
core_metrics_results = diversity.pipelines.core_metrics_phylogenetic(denoised_sequences.table,
                                                                     tree_placements.tree,
                                                                     sampling_depth = 2000,
                                                                     metadata = sample_metadata)
                                                                    

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1735: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.013386670794032449 and the largest is 5.24952559288162.
  RuntimeWarning
/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude 

### Question

Where did we get the value `2000` from? Why did we pick that?

# Alpha diversity


Alpha diversity asks whether the distribution of features within a sample (or groups of samples) differs between different conditions. The comparison makes no assumptions about the features that are shared between samples; two samples can have the same alpha diversity and not share any features. The rarefied alpha diversity produced by `diversity` is a univariate, continuous value and can be tested using common non-parametric statistical tests.

We can test our covariates of interest against Faith’s phylogenetic diversity and Pielou’s evenness value by running:

In [29]:
faith_pd_statistics = diversity.visualizers.alpha_group_significance(core_metrics_results.faith_pd_vector,
                                                                     sample_metadata)

In [30]:
faith_pd_statistics.visualization

<visualization: Visualization uuid: d97e8fbc-cbc2-4610-82a3-751aea42ea30>

In [31]:
evenness_statistics = diversity.visualizers.alpha_group_significance(core_metrics_results.evenness_vector,
                                                                     sample_metadata)

### Question

1. Is there a difference in evenness between genotype? Is there a difference in phylogenetic diversity between genotype?

2. Based on the group significance test, is there a difference in phylogenetic diversity by genotype? Is there a difference based on the donor?

If we had a continuous covariate that we thought was associated with the alpha diversity, we could test that using qiime diversity alpha-correlation. However, the only continuous variable in this dataset is the days_since_transplant.

In some experiments, multiple interacting factors may impact alpha diversity together. If our alpha diversity estimates follow a normal distribution, we may use analysis of variance (ANOVA) to test whether multiple effects significantly impact alpha diversity. This test is present in the `longitudinal` plugin:

In [32]:
faith_pd_anova = longitudinal.visualizers.anova(metadata = sample_metadata.merge(
                                                            core_metrics_results.faith_pd_vector.view(qiime2.Metadata)),
                                                 formula = 'faith_pd ~ genotype * donor_status')

In [33]:
faith_pd_anova.visualization

<visualization: Visualization uuid: 89a62927-105a-439d-a24e-84a35ee7ae24>

# Beta diversity

Next, we’ll compare the structure of the microbiome communities using beta diversity. Start by making a visual inspection of the principle coordinates plots (PCoA) plots that were generated by `emperor` and `core_metrics_results/weighted_unifrac_emperor.qzv`.

### Question

1. Open the unweighted UniFrac emperor plot (core-metrics-results/unweighted_unifrac_emperor.qzv) first. Can you find separation in the data? If so, can you find a metadata factor that reflects the separation? What if you used weighted UniFrac distance (`core-metrics-results/weighted_unifrac_emperor.qzv`)?

2. One of the major concerns in mouse studies is that sometimes differences in communities are due to natural variation in cages. Do you see clustering by cage?

Now, let’s analyze the statistical trends using PERMANOVA. [PERMANOVA](https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1442-9993.2001.01070.pp.x) tests the hypothesis that samples within a group are more similar to each other than they are to samples in another group. To put it another way, it tests whether the within-group distances from each group are different from the between group distance. We expect samples that are similar to have smaller distances from each other, so if our hypothesis that one group is different from another is true, we’d expect the within-group distances to be smaller than the between group distance.

Let’s use the `beta_group_significance` command to test whether the donor identity (which we qualitatively identified as a major separator in PCoA space) is associated with significant differences in weighted and unweighted UniFrac distance.

In [34]:
unweighted_unifrac_donor_significance = diversity.\
                                        visualizers.\
                                        beta_group_significance(core_metrics_results.\
                                                                unweighted_unifrac_distance_matrix,
                                                                sample_metadata.get_column('donor'))




In [35]:
unweighted_unifrac_donor_significance.visualization

<visualization: Visualization uuid: c1a03e02-e82b-46ac-b7a6-350e21be7d6f>

In [37]:
weighted_unifrac_donor_significance = diversity.\
                                        visualizers.\
                                        beta_group_significance(core_metrics_results.\
                                                                weighted_unifrac_distance_matrix,
                                                                sample_metadata.get_column('donor'))


In [38]:
weighted_unifrac_donor_significance.visualization

<visualization: Visualization uuid: a95d2829-bf69-4bd2-b124-7a9191c85aa2>

Let’s also check whether there’s a relationship between the cage in which a mouse lives and the beta diversity, since “cage effect” is often an important technical effect to consider. Since we have several cages, we’ll use the `pairwise` parameter that will let us check whether there are individual differences between the cages driving the difference. This may be useful, since if we check the metadata, we may find that cage is nested by donor.

In [39]:
unweighted_unifrac_cage_significance = diversity.\
                                       visualizers.\
                                       beta_group_significance(core_metrics_results.\
                                                               unweighted_unifrac_distance_matrix,
                                                               sample_metadata.get_column('cage_id'),
                                                               pairwise = True)

In [40]:
unweighted_unifrac_cage_significance.visualization


<visualization: Visualization uuid: 2912ef0b-1537-4793-a4cc-712e96af5977>

In [41]:
weighted_unifrac_cage_significance = diversity.\
                                    visualizers.\
                                    beta_group_significance(core_metrics_results.\
                                                            weighted_unifrac_distance_matrix,
                                                            sample_metadata.get_column('cage_id'),
                                                            pairwise = True)


In [42]:
weighted_unifrac_cage_significance.visualization

<visualization: Visualization uuid: 0aeb0fd8-7568-4781-95ff-3bb57ff7542d>

### Question

1. Is there a significant effect of donor?

2. From the metadata, we know that cage C31, C32, and C42 all house mice transplanted from one donor, and that cages C43, C44, and C49 are from the other. Is there a significant difference in the microbial communities between samples collected in cage C31 and C32? How about between C31 and C43? Do the results look the way you expect, based on the boxplots for donor?

A significant difference in PERMANOVA can reflect a large difference between the group or differences in variances within a group. This means that we might see a statistically significant difference even if it’s caused by variation within one group. Distance boxplots can help give a visual sense of this, but it’s nice to use a statistical test to confirm this. We can use the [permdisp](https://pubmed.ncbi.nlm.nih.gov/16706913/) to help rule out differences due to a high degree of dispersion (within-group variance) in one of the groups of interest.

We can specify that we want to use permdisp using the `method` flag in `qiime diversity beta_group_significance`. Let’s explore dispersion based on `cage_id` to check whether are cage-related differences are due to large within-cage variance.

In [44]:
unweighted_unifrac_cage_significance_disp = diversity.\
                                            visualizers.\
                                            beta_group_significance(core_metrics_results.weighted_unifrac_distance_matrix,
                                                                    sample_metadata.get_column('cage_id'),
                                                                    method = 'permdisp',
                                                                    pairwise = True)

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.029200949163027526 and the largest is 0.6113748299377452.
  RuntimeWarning
/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes secti

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0034645186000409815 and the largest is 0.1260720132266575.
  RuntimeWarning


In [45]:
unweighted_unifrac_cage_significance_disp = diversity.\
                                        visualizers.\
                                            beta_group_significance(core_metrics_results.weighted_unifrac_distance_matrix,
                                                                    sample_metadata.get_column('cage_id'),
                                                                    pairwise = True)

In [46]:
unweighted_unifrac_cage_significance_disp.visualization

<visualization: Visualization uuid: 2d9f9f7c-3448-4eed-9fa6-cfd0310f865e>

### Question

Is there a significant difference in variance for any of the cages?

We can also use the `adonis` action to look at a multivariate model. The adonis action uses a PERMANOVA test, but a different implementation that permits multiple effects to be tested simultaneously (similar to how we used ANOVA earlier for multivariate effects on alpha diversity). Let’s look at the intersection between donor and genotype.

In [ ]:
#####qiime diversity adonis \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/unweighted_adonis.qzv \
  --p-formula genotype+donor
    
testuj!!!@

In [48]:
unweighted_unifrac_cage_significance_disp = diversity.\
                                            visualizers.\
                                            adonis(core_metrics_results.weighted_unifrac_distance_matrix,
                                                                    sample_metadata.get_column('cage_id'),
                                                                    formula = 'genotype+donor')

TypeError: Parameter 'metadata' received <CategoricalMetadataColumn name='cage_id' id_count=48> as an argument, which is incompatible with parameter type: Metadata

In [49]:
unweighted_unifrac_cage_significance_disp = diversity.\
                                            visualizers.\
                                            adonis(core_metrics_results.weighted_unifrac_distance_matrix,
                                                   sample_metadata,
                                                   formula = 'genotype+donor')

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /tmp/tmpakcegk8r/dm.tsv /tmp/tmpakcegk8r/md.tsv genotype+donor 999 1 /tmp/qiime2-temp-dpm2der_/adonis.tsv



### Question

1. Is there a significant effect of donor?

2. From the metadata, we know that cage C31, C32, and C42 all house mice transplanted from one donor, and that cages C43, C44, and C49 are from the other. Is there a significant difference in the microbial communities between samples collected in cage C31 and C32? How about between C31 and C43? Do the results look the way you expect, based on the boxplots for donor?

3. If you adjust for donor in the adonis model, do you retain an effect of genotype? What percentage of the variation does genotype explain?

# Taxonomic classification


Up until now we have been performing diversity analyses directly on ASVs; in other words, we have assessed the similarity between samples based purely on the unique sequence variants that were observed in each sample. In most experiments we would like to get a sense of what microbial taxa are present — to identify ASVs and give them “names”. To do this, we’ll use the `feature_classifier` plugin to classify ASVs taxonomically.

For this analysis, we’ll use a pre-trained naive Bayes machine-learning classifier that was trained to differentiate taxa present in the 99% Greengenes 13_8 reference set trimmed to 250 bp of the V4 hypervariable region (corresponding to the 515F-806R primers). [This classifier works](https://microbiomejournal.biomedcentral.com/articles/10.1186/s40168-018-0470-z) by identifying k-mers that are diagnostic for particular taxonomic groups, and using that information to predict the taxonomic affiliation of each ASV. We can download the pre-trained classifier here:

In [50]:
!wget -O $workdir/"nb_classifier.qza" \
  "https://data.qiime2.org/2020.2/common/gg-13-8-99-515-806-nb-classifier.qza"

--2020-09-06 19:09:15--  https://data.qiime2.org/2020.2/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2020.2/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2020-09-06 19:09:16--  https://s3-us-west-2.amazonaws.com/qiime2-data/2020.2/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.176.88
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.176.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28373581 (27M) [application/x-www-form-urlencoded]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//nb_classifier.qza’

/home/user/Document 100%[===================>]  27.06M  1022KB/s    in 29s     

2020-09-06 19:09:4

In [51]:
gg_13_8_99_515_806 = qiime2.Artifact.load(workdir+'nb_classifier.qza')

It’s worth noting that Naive Bayes classifiers perform best when they’re trained for the specific hypervariable region amplified. You can train a classifier specific for your dataset based on the [training classifiers tutorial](https://docs.qiime2.org/2020.6/tutorials/feature-classifier/) or download classifiers for other datasets from the [QIIME 2 resource page](https://docs.qiime2.org/2020.6/data-resources/). Classifiers can be re-used for consistent versions of the underlying packages, database, and region of interest.

In [52]:
taxonomy = feature_classifier.methods.classify_sklearn(denoised_sequences.representative_sequences,
                                                       gg_13_8_99_515_806)

Now, let’s review the taxonomy associated with the sequences using the `metadata.visualizers.tabulate`.

In [53]:
seq_taxonomy = metadata.visualizers.tabulate(taxonomy.classification.view(qiime2.Metadata))
seq_taxonomy.visualization

<visualization: Visualization uuid: 37bbb518-2954-455e-afde-04c999c3a080>

Let’s also tabulate the representative sequences `(FeatureData[Sequence])`. Tabulating the representative sequences will allow us to see the sequence assigned to the identifier and interactively blast the sequence against the NCBI database.

In [54]:
dada2_rep_set_visualization = feature_table.visualizers.tabulate_seqs(denoised_sequences.representative_sequences)

### Question

1. Find the feature, `07f183edd4e4d8aef1dcb2ab24dd7745`. What is the taxonomic classification of this sequence? What’s the confidence for the assignment?

2. How many features are classified as `g__Akkermansia`?

3 Use the tabulated representative sequences to look up these features. If you blast them against NCBI, do you get the same taxonomic identifier as you obtained with `feature_classifier`?

### Note

You might notice that some features do not have taxonomic assignments, which for the Greengenes database is indicated by a blank string at the level (e.g., "`g__`"). These indicate that there is not enough information for the Greengenes database to differentiate members of that clade, either due to ambiguity in the database or because the gene region being sequenced doesn’t provide the resolution to distinguish members of that clade. This is distinct from cases where `feature_classifier` cannot reliably classify the ASV to a deeper level: in those cases, an incomplete taxonomy string will be provided. Hence, you may see two different types of “underclassification” in your data: e.g., `k__Bacteria`; `p__Firmicutes; c__Clostridia; o__Clostridiales; f__Christensenellaceae; g__; s__` (genus and species annotations are missing in Greengenes) as well as `k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Christensenellaceae` (`feature_classifier` could not confidently classify that ASV at genus level).

### Note

You may also notice that more than one ASV has the same taxonomic assignment. This is normal — unique ASVs do not necessarily map to unique taxonomic groups! We visualize the frequency of each taxonomic group in barplots (as described below) or use the `taxa` plugin to `collapse` our feature table based on taxonomic affiliation.

# Taxonomy barchart

Since we saw a difference in diversity in this dataset, we may want to look at the taxonomic composition of these samples. To visualize this, we will build a taxonomic barchart of the samples we analyzed in the diversity dataset.

Before doing this, we will first filter out any samples with fewer features than our rarefaction threshold (`2000`). We can filter samples using the `feature_table` plugin with the `filter_samples` method. This lets us filter our table based on a variety of criteria such as the number of counts (frequency, `min_frequency` and `max_frequency`), number of features (`min_features` and `max_features`), or sample metadata (`where`). See the [filtering tutorial](https://docs.qiime2.org/2020.6/tutorials/filtering/) for more details and examples.

For this example, we need to filter out samples with fewer sequences than our rarefaction depth.

In [55]:
table_min_2k = feature_table.methods.filter_samples(denoised_sequences.table,
                                                     min_frequency = 2000)

Now, let’s use the filtered table to build an interactive barplot of the taxonomy in each sample.

In [56]:
taxa_barplot_seq = taxa.visualizers.barplot(table_min_2k.filtered_table,
                                            taxonomy.classification,
                                            sample_metadata)

### Question

Visualize the data at level 2 (phylum level) and sort the samples by donor, then by genotype. Can you observe a consistent difference in phylum between the donors? Does this surprising you? Why or why not?

# Differential abundance with ANCOM

Many microbiome investigators are interested in testing whether individual ASVs or taxa are more or less abundant in different sample groups. This is known as differential abundance. Microbiome data present several challenges for performing differential abundance using convential methods. Microbiome abundance data are inherently sparse (have a lot of zeros) and compositional (everything adds up to 1). Because of this, traditional statistical methods that you may be familiar with, such as ANOVA or t-tests, are not appropriate for performing differential abundance tests of microbiome data and lead to a high false-positive rate. ANCOM is a compositionally aware alternative that allows to test for differentially abundant features. If you’re unfamiliar with the technique, it’s worthwhile to review the [ANCOM paper](https://pubmed.ncbi.nlm.nih.gov/26028277/) to better understand the method.

Before we begin, we will filter out low abundance/low prevalence ASVs. Filtering can provide better resolution and limit false discovery rate (FDR) penalty on features that are too far below the noise threshhold to be applicable to a statistical test. A feature that shows up with 10 counts may be a real feature that is present only in that sample, may be a feature that’s present in several samples but only got amplified and sequenced in one sample because PCR is a somewhat stochastic process, or it may be noise. It’s not possible to tell, so feature-based analysis may be better after filtering low abundance features. However, filtering also shifts the composition of a sample, further disrupting the relationship. Here, the filtering is performed as a trade off between the model, computational efficiency, and statistical practicality.

In [57]:
table_2k_abund = feature_table.methods.filter_features(table_min_2k.filtered_table,
                                                      min_frequency = 50,
                                                      min_samples = 4)

ANCOM fundamentally operates on a `FeatureTable[Frequency]`, which contains the frequencies of features in each sample. However, ANCOM cannot tolerate zeros (because compositional methods typically use a log-transform or a ratio and you can’t take the log or divide by zeros). To remove the zeros from our table, we add a pseudocount to the `FeatureTable[Frequency]` Artifact, creating a `FeatureTable[Composition]` in its place.

In [58]:
table2k_abund_comp = composition.methods.add_pseudocount(table_2k_abund.filtered_table)

Let’s use ANCOM to check whether there is a difference in the mice based on their donor and then by their genetic background. The test will calculate the number of ratios between pairs of ASVs that are significantly different with FDR-corrected p < 0.05.

In [59]:
ancom_donor = composition.visualizers.ancom(table2k_abund_comp.composition_table,
                                            sample_metadata.get_column('donor'))
ancom_donor.visualization

<visualization: Visualization uuid: fff06d8c-db02-43a2-a7df-d9e4b2592ca4>

In [60]:
ancom_genotype = composition.visualizers.ancom(table2k_abund_comp.composition_table,
                                               sample_metadata.get_column('genotype'))
ancom_genotype.visualization

<visualization: Visualization uuid: 7bf1c1df-45d1-440e-89db-1eb27cc4b3c9>

When you open the ANCOM visualizations, you’ll see a [volcano plot](https://en.wikipedia.org/wiki/Volcano_plot_(statistics) on top, which relates the ANCOM W statistic to the CLR (center log transform) for the groups. The W statistic is the number of ANCOM subhypotheses that have passed for each individual taxon, indicating that the ratios of that taxon’s relative abundance to the relative abundances of W other taxa were detected to be significantly different (typically FDR-adjusted p < 0.05). Because differential abundance in ANCOM is based on the ratio between tests, it does not produce a traditional p-value.

### Question

Open the ANCOM visualizations for the donor and genotype and the taxonomy visualization artifact.

1. Are there more differentially abundant features between the donors or the mouse genotype? Did you expect this result based on the beta diversity?

2. Are there any features that are differentially abundant in both the donors and by genotype?

3. How many differentially abundant features are there between the two genotypes? Using the percentile abundances and volcano plot as a guide, can you tell if they are more abundant in wild type or susceptible mice?

4. Use taxonomy metadata visualization and search sequence identifiers for the significantly different features by genotype. What genera do they belong to?

# Taxonomic classification again

It is possible to [increase taxonomic classification accuracy](https://www.biorxiv.org/content/10.1101/406611v2) by showing the taxonomic classifier what a typical animal stool sample looks like before attempting classification. To do that we will have to retrain the naive Bayes classifier. Fortunately, a representation of a typical stool sample that is derived from [Qiita](https://qiita.ucsd.edu/) data is available from the [readytowear collection](https://github.com/BenKaehler/readytowear).

If you feel that these samples are not typical stool samples, it is possible to, for instance, assemble data on just mouse or just human (or just human and mouse) stool samples using [q2_clawback](https://library.qiime2.org/plugins/q2-clawback/7/). We will not attempt that here because it takes a while to run, but details are available in the [tutorial](https://forum.qiime2.org/t/using-q2-clawback-to-assemble-taxonomic-weights/5859).

Start by downloading the stool data, along with the 99% Greengene 13_8 reference data.

In [61]:
!wget -O $workdir/"ref_seqs_v4.qza" \
  "https://data.qiime2.org/2020.6/tutorials/pd-mice/ref_seqs_v4.qza"

--2020-09-06 19:21:30--  https://data.qiime2.org/2020.6/tutorials/pd-mice/ref_seqs_v4.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/ref_seqs_v4.qza [following]
--2020-09-06 19:21:31--  https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/ref_seqs_v4.qza
Resolving qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)... 52.218.244.89
Connecting to qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)|52.218.244.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9319426 (8.9M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//ref_seqs_v4.qza’

/home/user/Document 100%[===================>]   8.89M   933KB/s    in 9.9s    

2020-09-06 19:21

In [62]:
!wget -O $workdir/"ref_tax.qza" \
  "https://data.qiime2.org/2020.6/tutorials/pd-mice/ref_tax.qza"

--2020-09-06 19:21:42--  https://data.qiime2.org/2020.6/tutorials/pd-mice/ref_tax.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/ref_tax.qza [following]
--2020-09-06 19:21:43--  https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/ref_tax.qza
Resolving qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)... 52.218.224.73
Connecting to qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)|52.218.224.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2604632 (2.5M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//ref_tax.qza’

/home/user/Document 100%[===================>]   2.48M   776KB/s    in 3.3s    

2020-09-06 19:21:47 (776 KB/s) -

In [63]:
!wget -O $workdir/"animal_distal_gut.qza" \
  "https://data.qiime2.org/2020.6/tutorials/pd-mice/animal_distal_gut.qza"

--2020-09-06 19:21:48--  https://data.qiime2.org/2020.6/tutorials/pd-mice/animal_distal_gut.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/animal_distal_gut.qza [following]
--2020-09-06 19:21:48--  https://qiime2-data.s3-us-west-2.amazonaws.com/2020.6/tutorials/pd-mice/animal_distal_gut.qza
Resolving qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)... 52.218.184.241
Connecting to qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)|52.218.184.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229276 (224K) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-mouse-tutorial//animal_distal_gut.qza’

/home/user/Document 100%[===================>] 223.90K   360KB/s    in 0

In [64]:
ref_seqs_v4 = qiime2.Artifact.load(workdir+'ref_seqs_v4.qza')

In [65]:
ref_tax = qiime2.Artifact.load(workdir+'ref_tax.qza')

In [66]:
animal_distal_gut = qiime2.Artifact.load(workdir+'animal_distal_gut.qza')

Next retrain the classifier.

In [67]:
classifier = feature_classifier.actions.fit_classifier_naive_bayes(ref_seqs_v4,
                                                                ref_tax,
                                                                animal_distal_gut)

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/q2_feature_classifier/classifier.py:102: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.22.1. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


We can use the new classifier in exactly the same way as the standard classifier that we downloaded above.

In [74]:
bespoke_taxonomy = feature_classifier.actions.classify_sklearn(denoised_sequences.representative_sequences,
                                                              classifier.classifier)


In [83]:
filter_bespoke_taxonomy = metadata.visualizers.tabulate(bespoke_taxonomy.classification.view(qiime2.Metadata))

In [84]:
filter_bespoke_taxonomy.visualization

<visualization: Visualization uuid: 302127da-e50e-4fbb-bac6-893dbbec68f4>

### Question

Open up the old `taxonomy` visualization and compare it to the `bespoke_taxonomy` visualization.

1. Search for “ovatus” in both. Is there an ASV in the new taxonomy that wasn’t present in the original?

2. Revisit the `ancom_donor` visualization. Can you find that ASV?

When analyzing ANCOM results, it is possible to trace the ASVs that we found using the taxonomies that we have created. It is also possible to run ANCOM directly on taxonomic groups that we have discovered in our samples by counting features according to taxonomic classification. This has the advantage of pooling feature counts across taxonomically similar ASVs, for instance allowing exact species substitution between samples. The output is also more readable. On the down side, it has all the inaccuracies that come with automated taxonomic classification.

We will run through the pipeline twice, once with our original taxonomy and once with the new taxonomy, for the purpose of comparison. First using the original taxonomy:

In [85]:
uniform_table = taxa.actions.collapse(table_min_2k.filtered_table,
                                      taxonomy.classification,
                                      level = 7)
                                      

In [86]:
filtered_uniform_table = feature_table.actions.filter_features(uniform_table.collapsed_table,
                                                               min_frequency = 50,
                                                               min_samples = 4)

In [87]:
cfu_table = composition.actions.add_pseudocount(filtered_uniform_table.filtered_table)

In [88]:
ancom_donor_uniform = composition.actions.ancom(cfu_table.composition_table,
                                               sample_metadata.get_column('donor'))
ancom_donor_uniform.visualization

<visualization: Visualization uuid: a76a8d9a-b7e9-487e-a7e0-22e115af9524>

Now redo with the new taxonomy:

In [89]:
collapsed_bespoke_table = taxa.actions.collapse(table_min_2k.filtered_table,
                                               bespoke_taxonomy.classification,
                                                level = 7)

In [90]:
filtered_bespoke_table = feature_table.actions.filter_features(uniform_table.collapsed_table,
                                                               min_frequency = 50,
                                                               min_samples = 4)


In [91]:
cfb_table = composition.actions.add_pseudocount(filtered_bespoke_table.filtered_table)

In [92]:
ancom_donor_bespoke = composition.actions.ancom(cfb_table.composition_table,
                                               sample_metadata.get_column('donor'))
ancom_donor_bespoke.visualization

<visualization: Visualization uuid: b09f4e23-9ea7-4038-a5f3-443801fe03c3>

### Question

Compare final ANCOM visualizations. They are fairly similar, which is good.

1. Is Bacteroides ovatus present in the ANCOM results derived from our original taxonomy?

2. Is B. ovatus present in the new ANCOM results?

3. Why is that?

# Longitudinal analysis

This study includes a longitudinal component; samples from each mouse were collected 7, 14, 21, and 49 days post fecal transplant. We can use the `longitudinal` plug-in to explore the hypothesis that a mouse’s genetic background affected the change in the microbial community of each mouse. For this longitudinal analysis, we’re going to focus on beta diversity. Alpha diversity changes wildly in infants, but it’s often stable in adults over short time periods. We’re dealing with an adult fecal community over a relatively short time period, and there is no difference in alpha diversity with time. The [longitudinal analysis tutorial](https://docs.qiime2.org/2020.6/tutorials/longitudinal/) is an excellent resource for exploring longitudinal analyses of microbiome samples.



## PCoA-based analyses

We can start by exploring temporal change in the PCoA using the animations tab.

### Question

1. Open the unweighted UniFrac emperor plot and color the samples by mouse id. Click on the “animations” tab and animate using the day_post_transplant as your gradient and mouse_id as your trajectory. Do you observe any clear temporal trends based on the PCoA?

2. What happens if you color by day_post_transplant? Do you see a difference based on the day? Hint: Trying changing the colormap to a sequential colormap like viridis.

A volatility plot will let us look at patterns of variation along principle coordinate axes starting from the same point. This can be helpful since inter-individual variation can be large and this visualizations lets us focus instead on magnitude of change in each group and in each individual.

Let’s use the `longitudinal` plugin to look at how samples from an individual move along each PC. The `sample_metadata` column can take several types, including a metadata file (like our `metadata.tsv`) as well as a sample_metadata`[AlphaDiversity]`, sample_metadata`[Distance]` (files “viewable” as metadata), or a `PCoA` artifact.

In [93]:
pc_vol = longitudinal.actions.volatility(sample_metadata.merge(core_metrics_results.\
                                                               unweighted_unifrac_pcoa_results.\
                                                               view(qiime2.Metadata)),
                                                               state_column = 'days_post_transplant',
                                                               individual_id_column = 'mouse_id',
                                                               default_group_column = 'donor_status',
                                                               default_metric = 'Axis 2')

In [94]:
pc_vol.visualization

<visualization: Visualization uuid: 51d7c74e-ff3e-4a62-81f6-0ea9c2da7ac1>

### Question

Using the controls, look at variation in cage along PCs 1, 2, and 3. What kind of patterns do you see with time along each axis?

# Distance-based analysis

Now, let’s try looking directly at the pairwise distances between samples. Here, we’ll test the hypothesis that genotype affects the magnitude of change in the distance from the first sample collected from each mouse (7 days post transplant). We assume that given the rate of turnover in a microbial community, we might expect to see a change in the community over time. However, here we’ll ask if these changes are associated with host genotype.

We’ll start this analysis by looking at how much the microbial community of each mouse changes from 7 days post transplant. We use the `baseline` parameter to specify a static time point against which all other time points are compared; if we remove this parameter from the command, we look instead at the rate of change for each individual between each time point. See the [longitudinal analysis tutorial](https://docs.qiime2.org/2020.6/tutorials/longitudinal/) for more details.

In [95]:
from_first_unifrac = longitudinal.actions.first_distances(core_metrics_results.\
                                                          unweighted_unifrac_distance_matrix,
                                                          metadata = sample_metadata,
                                                          state_column = 'days_post_transplant',
                                                          individual_id_column = 'mouse_id',
                                                          baseline = 7)

We can again use volatility analysis to visualize the change in beta diversity based on distance.

In [96]:
from_first_unifrac_vol = longitudinal.actions.volatility(sample_metadata.\
                                                         merge(from_first_unifrac.first_distances.\
                                                               view(qiime2.Metadata)),
                                                         state_column = 'days_post_transplant',
                                                         individual_id_column = 'mouse_id',
                                                         default_metric = 'Distance',
                                                         default_group_column = 'donor_status')
                                                    
from_first_unifrac_vol.visualization                            

<visualization: Visualization uuid: 40e932b5-02c8-4666-b683-92f35a58ab90>

### Question

Based on the volatility plot, does one donor change more over time than the other? What about by genotype? Cage?


A linear mixed effects (LME) model lets us test whether there’s a relationship between a dependent variable and one or more independent variables in an experiment using repeated measures. Since we’re interested in genotype, we should use this as an independent predictor.

For our experiment, we’re currently interested in the change in distance from the initial timepoint, so we’ll use this as our outcome variable (given by `metric`).

The `linear_mixed_effects` action also requires a state column (`state_column`) which designates the time component in the metadata, and an individual identifier (`individual_id_column`). Which columns should we use in our data?

We can build a model either using the `formula` parameter or the `group_columns` parameter. For this analysis, we’re interested in whether genotype affects the longitudinal change in the microbial community. However, we also know from our cross sectional analysis that donor plays a large role in shaping the fecal community. So, we should also probably include that in this analysis. We may also want to consider cage effect in our experiment, since this is a common confounder in rodent studies. However, the original experimental design here was clever: although cages were grouped by donor (mice are coprophagic), they were of mixed genotype. This partial randomization helps limit some of the cage effects we might otherwise see.

Based on the experimental design, what group columns should we choose?

In [97]:
from_first_unifrac_lme = longitudinal.actions.linear_mixed_effects(sample_metadata.merge(from_first_unifrac.first_distances.view(qiime2.Metadata)),
                                                                   state_column = 'days_post_transplant',
                                                                   individual_id_column = 'mouse_id',
                                                                   metric = 'Distance',
                                                                   group_columns = 'genotype,donor')

from_first_unifrac_lme.visualization                

/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/user/miniconda2/envs/qiime2-2020.2/lib/python3.6/site-packages/q2_longitudinal/_longitudinal.py:291: UserWarning: This is only a warning, and the results of this action are still valid. The column name "predicted Distance" already exists in your metadata file. Any "raw" metadata that can be downloaded from the resulting visualization will contain overwritten values for this metadata column, not the original values.
  warnings.warn(warning, UserWarning)


<visualization: Visualization uuid: 8a2f7b0e-0f12-46c4-911d-07f8a266a962>

Now, let’s look at the results of the models

### Question

1. Is there a significant association between the genotype and temporal change?

2. Which genotype is more stable (has lower variation)?

3. Is there a temporal change associated with the donor? Did you expect or not expect this based on the volatility plot results?

4. Can you find an interaction between the donor and genotype?

### Note

Importantly, LME models also allow us to distinguish between two types of independent variables: fixed effects (e.g., experimental treatments) and random effects (e.g., biological factors that cannot be controlled in the experiment). By default, the `linear_mixed_effects` action in `longitudinal` uses the `individual_id_column` as a random effect, since we can expect that biological differences between individual subjects will impact the baseline values of the dependent variable we are testing. The rate of change — slope — is another inter-individual effect that we often might want to consider as a random effect in longitudinal experiments. See the [longitudinal analysis](https://docs.qiime2.org/2020.6/tutorials/longitudinal/) tutorial for more details and discussion of LME tests and effect types.

# Machine-learning classifiers for predicting sample characteristics

As an alternative (or complementary) approach to the methods we have used in this tutorial for testing if and how samples are different from one another, we can utilize [machine-learning methods](https://docs.qiime2.org/2020.6/tutorials/sample-classifier/) to determine how predictive microbiome composition is of other characteristics about a sample. For example, we may use machine-learning classifiers to predict a patient’s susceptibility to disease, or predict the treatment group that a sample belongs to. Additionally, many machine-learning methods report which features are most important for predicting sample characteristics, making this a useful approach for determining which features (ASVs, species, etc) are associated with a particular treatment, disease state, or other category of interest. All of this and much more can be found in the `sample_classifier` plugin. Here we will use this plugin to predict each mouse’s genotype and donor status based on their ASV composition using a Random Forest classifier (this pipeline can access many different machine-learning methods via the `estimator parameter`, but Random Forest classifiers are used by default).

In [98]:
sample_classifier_results = sample_classifier.actions.classify_samples(denoised_sequences.table,
                                                                       sample_metadata.\
                                                                       get_column('genotype_and_donor_status'),
                                                                       random_state = 666,
                                                                       n_jobs = 1)

In [99]:
sample_classifier_results.accuracy_results

<visualization: Visualization uuid: c448745e-9121-4af9-967f-5cdc2cabb0e4>

This pipeline generates a number of output artifacts and visualizations. You can read more about these in the [sample classifier](https://docs.qiime2.org/2020.6/tutorials/sample-classifier/) tutorial but right now let’s just focus on `sample_classifier_results` This visualization tells you how well your sample classifier performed via a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) and accompanying table of accuracy scores. This tells you how frequently each sample type is classified to each sample class, including the correct class label. Overall error rates are also reported in the table below.

### Question

How did we do? Just for fun, try predicting some of the other metadata columns to see how easily `cage_id` and other columns can be predicted.

Looks like we did pretty well! So we can see what features are most predictive of each sample class (donor and genotype groups). The importance scores are stored in the ./sample-classifier-results/feature_importance.qza artifact (pro tip: this can be view with the `metadata.visualizers.tabulate` command we covered earlier). Here we will generate a heatmap showing the mean abundance of the 100 most important ASVs in each genotype and donor group.

In [100]:
filtered_table_heatmap = sample_classifier.actions.heatmap(denoised_sequences.table,
                                                           sample_classifier_results.feature_importance,
                                                           sample_metadata.get_column('genotype_and_donor_status'),
                                                           group_samples = True,
                                                           feature_count = 100)

### Question

What features appear to differentiate genotypes? What about donors? Are any ASVs specific to a single sample group?

# Synthesis

Based on the results of the analysis, we can say that there is a difference in the microbial communities of these mice based on their donor and genetic background. (This recapitulates the results of the original analysis.)

We found that the donor is the primary driver of alpha diversity.

But, we saw differences by donor and genotype based on beta diversity. Using the PCoA emperor plots, we can see clear separation between the mice from the two donors. After adjusting for the donor, we saw a significant difference between the genotypes.

Although there wasn’t a clear pattern in the barchart at the phylum level between donors or genotypes, we were still able to find ASVs which differentiated the genotypes using ANCOM and Random Forest classification. There was no overlap between these ASVs in the donor and genetic background, supporting the hypothesis that the difference due to genotype is separate from the difference due to donor.

The volatility plots and temporal analysis showed that the microbiome in different genetic backgrounds changed differently over time.

This suggests that there is a genotype-specific effect on the microbiome of mice receiving fecal transplants.

💩🐁